In [3]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [4]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
##종웅님 
headers = {"X-Naver-Client-Id": "EmdlUXsc4CuKNm_kxYKS",
           "X-Naver-Client-Secret":"dSLoIEmMgy"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [5]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "건강측정용품", "param": [ "50000063"]},
          {"name": "구강위생용품", "param": [ "50000072"]},
          {"name": "눈건강용품", "param": [ "50000073"]},
          {"name": "문구/사무용품", "param": [ "50000158"]},
          {"name": "반려동물", "param": [ "50000155"]},
          {"name": "생활용품", "param": [ "50000078"]},
          {"name": "세탁용품", "param": [ "50000062"]},
          {"name": "수납/정리용품", "param": [ "50000076"]},
          {"name": "욕실용품", "param": [ "50000157"]},
          {"name": "자동차용품", "param": [ "50000055"]},
          {"name": "주방용품", "param": [ "50000061"]},
          {"name": "화방용품", "param": [ "50000054"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category":[
      {"name": "생활/건강", "param": [ "50000008"]},
      {"name": "공구", "param": [ "50000165"]},
    ],
    "device":i,
    "gender":"",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "생활/건강")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "공구")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["생활/건강"], axis=1, inplace=True)
df_all_a

,공구,건강측정용품,구강위생용품,눈건강용품,문구/사무용품,반려동물,생활용품,세탁용품,수납/정리용품,욕실용품,자동차용품,주방용품,화방용품,유형,성별,연령
period,,,,,,,,,,,,,,,,
2021-05-10,2.59407,0.21690,0.19453,0.04342,1.69198,1.30746,1.36762,0.09413,0.40601,0.82984,2.16440,3.27705,0.11265,PC+MO,전체,전체
2021-05-11,2.50434,0.21512,0.18086,0.04107,1.65956,1.24959,1.30383,0.08742,0.37992,0.77397,2.05288,3.08327,0.11340,PC+MO,전체,전체
2021-05-12,2.47263,0.21083,0.17260,0.04132,1.54470,1.20965,1.29076,0.08442,0.38454,0.76148,2.03787,2.94417,0.10793,PC+MO,전체,전체
2021-05-13,2.35484,0.21196,0.16222,0.03755,1.39914,1.15453,1.26463,0.08491,0.37275,0.71585,1.97642,2.80112,0.10184,PC+MO,전체,전체
2021-05-14,2.11727,0.19921,0.14242,0.03541,1.22780,1.10215,1.19559,0.07560,0.39289,0.65340,1.91408,2.58025,0.09210,PC+MO,전체,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,1.69290,0.03898,0.16042,0.02856,1.05970,0.97441,1.03973,0.07594,0.25524,0.61755,1.48561,2.15816,0.07728,MO,전체,전체
2023-06-30,1.51963,0.03608,0.14093,0.02490,0.95184,0.87170,0.91431,0.06467,0.21394,0.53197,1.34695,1.87932,0.06845,MO,전체,전체
2023-07-01,1.41694,0.04058,0.15804,0.02800,0.95810,0.97611,1.05171,0.08685,0.24666,0.61256,1.47791,2.12326,0.06702,MO,전체,전체


In [6]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "건강측정용품", "param": [ "50000063"]},
          {"name": "구강위생용품", "param": [ "50000072"]},
          {"name": "눈건강용품", "param": [ "50000073"]},
          {"name": "문구/사무용품", "param": [ "50000158"]},
          {"name": "반려동물", "param": [ "50000155"]},
          {"name": "생활용품", "param": [ "50000078"]},
          {"name": "세탁용품", "param": [ "50000062"]},
          {"name": "수납/정리용품", "param": [ "50000076"]},
          {"name": "욕실용품", "param": [ "50000157"]},
          {"name": "자동차용품", "param": [ "50000055"]},
          {"name": "주방용품", "param": [ "50000061"]},
          {"name": "화방용품", "param": [ "50000054"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category":[
      {"name": "생활/건강", "param": [ "50000008"]},
      {"name": "공구", "param": [ "50000165"]},
    ],
    "device":i,
    "gender":"f",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "생활/건강")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "공구")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["생활/건강"], axis=1, inplace=True)
df_all_b

,공구,건강측정용품,구강위생용품,눈건강용품,문구/사무용품,반려동물,생활용품,세탁용품,수납/정리용품,욕실용품,자동차용품,주방용품,화방용품,유형,성별,연령
period,,,,,,,,,,,,,,,,
2021-05-10,1.62852,0.24677,0.39659,0.08160,3.66285,2.95031,2.30259,0.20613,0.87982,1.32712,1.12253,7.76300,0.25514,PC+MO,여성,전체
2021-05-11,1.55544,0.23539,0.36841,0.07312,3.56032,2.82753,2.14107,0.18740,0.81432,1.24694,1.06489,7.27285,0.24991,PC+MO,여성,전체
2021-05-12,1.53491,0.22049,0.34831,0.07298,3.27679,2.71928,2.09171,0.18307,0.82960,1.20436,1.07874,6.93692,0.24485,PC+MO,여성,전체
2021-05-13,1.44602,0.22756,0.32278,0.06432,2.90175,2.57918,2.03047,0.18483,0.80568,1.14496,1.07522,6.54563,0.22904,PC+MO,여성,전체
2021-05-14,1.29683,0.20693,0.28116,0.05658,2.50479,2.43319,1.91519,0.16012,0.84065,1.03319,0.99813,6.03742,0.20566,PC+MO,여성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,1.21128,0.08535,0.39791,0.07039,2.62045,2.54973,2.36312,0.18486,0.62406,1.18751,1.02485,5.60551,0.20756,MO,여성,전체
2023-06-30,1.05225,0.07530,0.34528,0.06061,2.34024,2.25520,2.04350,0.15891,0.52864,1.03930,0.90828,4.86321,0.18657,MO,여성,전체
2023-07-01,1.08429,0.08241,0.38351,0.06734,2.34029,2.51892,2.38757,0.21229,0.60984,1.19060,1.10345,5.53184,0.17912,MO,여성,전체


In [7]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "건강측정용품", "param": [ "50000063"]},
          {"name": "구강위생용품", "param": [ "50000072"]},
          {"name": "눈건강용품", "param": [ "50000073"]},
          {"name": "문구/사무용품", "param": [ "50000158"]},
          {"name": "반려동물", "param": [ "50000155"]},
          {"name": "생활용품", "param": [ "50000078"]},
          {"name": "세탁용품", "param": [ "50000062"]},
          {"name": "수납/정리용품", "param": [ "50000076"]},
          {"name": "욕실용품", "param": [ "50000157"]},
          {"name": "자동차용품", "param": [ "50000055"]},
          {"name": "주방용품", "param": [ "50000061"]},
          {"name": "화방용품", "param": [ "50000054"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category":[
      {"name": "생활/건강", "param": [ "50000008"]},
      {"name": "공구", "param": [ "50000165"]},
    ],
    "device":i,
    "gender":"m",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "생활/건강")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "공구")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["생활/건강"], axis=1, inplace=True)
df_all_c

,공구,건강측정용품,구강위생용품,눈건강용품,문구/사무용품,반려동물,생활용품,세탁용품,수납/정리용품,욕실용품,자동차용품,주방용품,화방용품,유형,성별,연령
period,,,,,,,,,,,,,,,,
2021-05-10,15.27294,0.39603,0.55906,0.10660,3.65201,2.42796,3.72283,0.21983,0.95694,2.85713,13.47036,6.97943,0.24416,PC+MO,남성,전체
2021-05-11,14.78331,0.35572,0.51036,0.10609,3.54331,2.24311,3.48019,0.20606,0.88135,2.61662,12.69351,6.52218,0.25067,PC+MO,남성,전체
2021-05-12,14.58016,0.34538,0.47478,0.10297,3.33651,2.16600,3.54243,0.19441,0.90402,2.58628,12.45668,6.28988,0.23521,PC+MO,남성,전체
2021-05-13,13.87722,0.33373,0.45082,0.09439,3.05288,2.01801,3.46033,0.19093,0.85288,2.36524,11.69875,5.93172,0.22478,PC+MO,남성,전체
2021-05-14,12.47838,0.30252,0.37920,0.09155,2.70334,1.81099,3.19198,0.17201,0.87555,2.12660,10.67609,5.41050,0.20039,PC+MO,남성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,14.92298,0.15753,0.52760,0.08620,3.22035,2.24655,3.87827,0.22142,0.83304,2.73775,13.19730,6.02583,0.20010,MO,남성,전체
2023-06-30,13.47494,0.14988,0.46346,0.07312,2.87480,2.03506,3.46587,0.17652,0.70092,2.44771,11.95059,5.22715,0.16676,MO,남성,전체
2023-07-01,12.22199,0.17740,0.53623,0.08782,2.91353,2.40288,3.97131,0.26413,0.83071,2.94359,12.72415,5.95249,0.17560,MO,남성,전체


In [8]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7065 entries, 2021-05-10 to 2023-07-03
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   공구       7065 non-null   float64
 1   건강측정용품   7065 non-null   float64
 2   구강위생용품   7065 non-null   float64
 3   눈건강용품    7065 non-null   float64
 4   문구/사무용품  7065 non-null   float64
 5   반려동물     7065 non-null   float64
 6   생활용품     7065 non-null   float64
 7   세탁용품     7065 non-null   float64
 8   수납/정리용품  7065 non-null   float64
 9   욕실용품     7065 non-null   float64
 10  자동차용품    7065 non-null   float64
 11  주방용품     7065 non-null   float64
 12  화방용품     7065 non-null   float64
 13  유형       7065 non-null   object 
 14  성별       7065 non-null   object 
 15  연령       7065 non-null   object 
dtypes: float64(13), object(3)
memory usage: 938.3+ KB


In [9]:
## 엑셀에 저장
df_all.to_excel("MCLS_05_all_fm_all.xlsx", sheet_name="click ratio")
df_all.head()

,공구,건강측정용품,구강위생용품,눈건강용품,문구/사무용품,반려동물,생활용품,세탁용품,수납/정리용품,욕실용품,자동차용품,주방용품,화방용품,유형,성별,연령
period,,,,,,,,,,,,,,,,
2021-05-10,2.59407,0.21690,0.19453,0.04342,1.69198,1.30746,1.36762,0.09413,0.40601,0.82984,2.16440,3.27705,0.11265,PC+MO,전체,전체
2021-05-11,2.50434,0.21512,0.18086,0.04107,1.65956,1.24959,1.30383,0.08742,0.37992,0.77397,2.05288,3.08327,0.11340,PC+MO,전체,전체
2021-05-12,2.47263,0.21083,0.17260,0.04132,1.54470,1.20965,1.29076,0.08442,0.38454,0.76148,2.03787,2.94417,0.10793,PC+MO,전체,전체
2021-05-13,2.35484,0.21196,0.16222,0.03755,1.39914,1.15453,1.26463,0.08491,0.37275,0.71585,1.97642,2.80112,0.10184,PC+MO,전체,전체
2021-05-14,2.11727,0.19921,0.14242,0.03541,1.22780,1.10215,1.19559,0.07560,0.39289,0.65340,1.91408,2.58025,0.09210,PC+MO,전체,전체
